In [1]:
import collections
import json
import keras
import numpy as np
import os
import sys
sys.path.append("../../../ecg")
import scipy.stats as sst

import util
import load

/Users/wonderit/opt/anaconda3/envs/ecg/lib/python3.7/site-packages/h5py/__init__.py:73: UserWarning: h5py is running against HDF5 1.10.4 when it was built against 1.8.4, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.
/Users/wonderit/opt/anaconda3/envs/ecg/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/wonderit/opt/anaconda3/envs/ecg/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/wonderit/opt/anaconda3/envs/ecg/lib/python3.7/site-packages/tensorflow/p

In [2]:
model_path = "../../../saved/cinc17/1597729558-4/0.415-0.863-017-0.264-0.910.hdf5"
data_path = "../dev.json"

data = load.load_dataset(data_path)
preproc = util.load(os.path.dirname(model_path))
model = keras.models.load_model(model_path)

100%|██████████| 852/852 [00:00<00:00, 1159.20it/s]



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
data_path = "../train.json"
with open("../train.json", 'rb') as fid:
    train_labels = [json.loads(l)['labels'] for l in fid]
counts = collections.Counter(preproc.class_to_int[l[0]] for l in train_labels)
counts = sorted(counts.most_common(), key=lambda x: x[0])
counts = list(zip(*counts))[1]
smooth = 500
counts = np.array(counts)[None, None, :]
total = np.sum(counts) + counts.shape[1]
prior = (counts + smooth) / float(total)
##%
print(prior)

[[[0.15448743 0.66301941 0.34596848 0.09691286]]]


In [4]:
probs = []
labels = []
for x, y  in zip(*data):
    x, y = preproc.process([x], [y])
    probs.append(model.predict(x))
    # print(sst.mode(np.argmax(probs[0] / prior, axis=2).squeeze()))
    labels.append(y)

In [5]:
preds = []
ground_truth = []
for p, g in zip(probs, labels):
    preds.append(sst.mode(np.argmax(p / prior, axis=2).squeeze())[0][0])
#     preds.append(sst.mode(np.argmax(p, axis=2).squeeze())[0][0])
    ground_truth.append(sst.mode(np.argmax(g, axis=2).squeeze())[0][0])

In [6]:
gt = np.array(ground_truth)
print('0:', len(gt[gt == 0]))
print('1:', len(gt[gt == 1]))
print('2:', len(gt[gt == 2]))
print('3:', len(gt[gt == 3]))
print(preds)

0: 72
1: 486
2: 259
3: 35
[1, 3, 2, 0, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 0, 2, 3, 1, 2, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 2, 2, 1, 3, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 2, 1, 0, 0, 1, 1, 2, 2, 1, 2, 0, 1, 1, 3, 1, 1, 2, 1, 3, 2, 2, 0, 1, 2, 1, 2, 0, 0, 2, 0, 1, 1, 3, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 0, 2, 2, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 3, 3, 3, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 0, 2, 2, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 2, 0, 1, 2, 1, 2, 3, 1, 1, 1, 0, 2, 1, 2, 2, 1, 1, 1, 2, 1, 0, 1, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 0, 1, 1, 1, 3, 1, 1, 1, 1, 0, 1, 2, 1, 1, 3, 1, 1, 0, 1, 1, 1, 1, 2, 2, 1, 0, 1

In [7]:
import sklearn.metrics as skm
report = skm.classification_report(
            ground_truth, preds,
            target_names=preproc.classes,
            digits=3)
scores = skm.precision_recall_fscore_support(
                    ground_truth,
                    preds,
                    average=None)
print(report)
print("CINC Average {:3f}".format(np.mean(scores[2][:3])))

              precision    recall  f1-score   support

           A      0.821     0.889     0.853        72
           N      0.918     0.893     0.905       486
           O      0.803     0.834     0.818       259
           ~      0.688     0.629     0.657        35

    accuracy                          0.864       852
   macro avg      0.807     0.811     0.808       852
weighted avg      0.865     0.864     0.864       852

CINC Average 0.858875
